<a href="https://colab.research.google.com/github/Hernan4444/diplomado-sistemas-recomendadores/blob/master/Diplomado_Alumno_2019_Sistemas_Recomendadores_3_Content_Based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica de Sistemas Recomendadores 3: Content based

En este práctico, volveremos a utilizar la biblioteca de Python [sklearn](https://scikit-learn.org/stable/), para aprender sobre 2 algoritmos para recomendación basado en contenidos y de unas herramientas para preprocesar los textos. En particula, este practico verá:

* TF-IDF
* Word Embeddings
* Uso de Stop Words

**Ayudantes**: Manuel Cartagena, Andrés Carvallo y Patricio Cerda




# Actividad 1

Antes de empezar con la actividad, responder la siguiente pregunta con lo visto en clases

**Pregunta:** Explique con sus palabras a qué se refiere con recomendación basada en contenido. En particular responda

- ¿Qué datos se utilizan para recomendación basada en contenidos?
- Mencione un ejemplo donde sea factible utilizar este tipo de recomendación y justifique.





**Respuesta:** COMPLETAR

# Descargando la información

Vaya ejecutando cada celda presionando el botón de **Play** o presionando Ctrl+Enter (Linux y Windows) o Command+Enter (Macosx) para descargar las bases de datos.

*   Recursos:
  * `dictionary.p`
  * `dictionary-stemm.p`
  * `tfidf_model.p`
  * `tfidf_model-stemm.p`
*   Dataset:
  *  `corpus1.csv`

In [1]:
# Descarga de recursos
!curl -L -o 'resources.tar.gz' 'https://github.com/PUC-RecSys-Class/Syllabus/blob/master/Practico%204/files/resources.tar.gz?raw=true'

# Descompresión del archivo
!tar -xvf resources.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   157    0   157    0     0    447      0 --:--:-- --:--:-- --:--:--   447
100   168  100   168    0     0    268      0 --:--:-- --:--:-- --:--:--   268
100 1754k  100 1754k    0     0  1734k      0  0:00:01  0:00:01 --:--:-- 30.5M
resources/
resources/dictionary-stemm.p
resources/dictionary.p
resources/tfidf_model-stemm.p
resources/tfidf_model.p


In [2]:
# Descarga del dataset
!curl -L -o 'dataset.tar.gz' 'https://github.com/PUC-RecSys-Class/Syllabus/blob/master/Practico%204/files/dataset.tar.gz?raw=true'

# Descompresión del archivo
!tar -xvf dataset.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   155    0   155    0     0    462      0 --:--:-- --:--:-- --:--:--   464
100   166  100   166    0     0    264      0 --:--:-- --:--:-- --:--:--  162k
100 3117k  100 3117k    0     0  3403k      0 --:--:-- --:--:-- --:--:-- 3403k
./._corpus1.csv
corpus1.csv


# Revisar archivos descargados

Revisemos el _dataset_ descargado:


In [3]:
import pandas as pd

corpus_df = pd.read_csv('./corpus1.csv', sep='\t',
                        header=None, encoding='latin',
                        names=['id', 'title', 'abstract'])
corpus_df.head(5)

,id,title,abstract
0,100002,Nonlinear Shape Statistics in Mumford{Shah Bas...,We present a variational integration of nonlin...
1,100007,On the Complexity of Equilibria,"We prove complexity, approximability, and inap..."
2,100008,On QoS-Aware Publish-Subscribe,This position paper addresses the issue of sup...
3,10001,Checking Mergeable Priority Queues,We present an efficient algorithm which can ch...
4,100012,Mobile Code Security by Java Bytecode Instrume...,Mobile code provides significant opportunities...


Podemos ver que este _dataet: contiene 3 columnas:
* **_id_**: identificador de cada texto
* **_title_**: título del documento, en este caso, de un _paper_
* **_abstract_**: primer párafo del _paper_ que es una representación abreviada, objetiva y precisa del contenido de un documento o recurso, sin interpretación crítica y sin mención expresa del autor del resumen.

## Preparar entorno
Primero es necesario instalar algunas librerías previas

In [4]:
!pip install nltk
!pip install sklearn
!pip install gensim
!pip install pandas
!pip install numpy

Luego necesitamos importar las librerías a utilizar en este práctico. No se asusten por todas las librerías, iremos explicando lo más importante a medida que se avanza en el práctico.

In [0]:
import string

import gensim
import nltk
import numpy as np
import pandas as pd
import sklearn

from collections import Counter
from os.path import isfile
from textwrap import wrap

from gensim import corpora, models, similarities
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

import json
import warnings
import gensim 
warnings.filterwarnings("ignore")
pd.options.display.max_columns = None



## Preprocesamiento de datos

Volvemos a cargar el _dataset_ a utilizar

In [6]:
corpus_df = pd.read_csv('./corpus1.csv', sep='\t',
                        header=None, encoding='latin',
                        names=['id', 'title', 'abstract'])
corpus_df.head(5)

,id,title,abstract
0,100002,Nonlinear Shape Statistics in Mumford{Shah Bas...,We present a variational integration of nonlin...
1,100007,On the Complexity of Equilibria,"We prove complexity, approximability, and inap..."
2,100008,On QoS-Aware Publish-Subscribe,This position paper addresses the issue of sup...
3,10001,Checking Mergeable Priority Queues,We present an efficient algorithm which can ch...
4,100012,Mobile Code Security by Java Bytecode Instrume...,Mobile code provides significant opportunities...


Luego descargamos las librerías de NLTK necesarias:

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

En este momento estamos bajando un _tokenizador_ específico llamado [Punkt Sentence Tokenizer](https://kite.com/python/docs/nltk.tokenize.punkt). Este será usado a continuación para realizar una cierta tarea con los textos (no vamos a decir cual es porque una actividad es que comenten que hace dado unos ejemplos que mostramos c: ). 

Lo siguiente es implementar una función que transforme texto no estructurado a una lista de *tokens* procesados.

In [8]:
def get_tokens(text):
    # Pasar todo a minuscula
    lowers = text.lower()
    
    # Quitar puntuación
    no_punctuation = lowers.translate({ord(c): None for c in string.punctuation})
    
    # Tokenizar 
    tokens = nltk.word_tokenize(no_punctuation)
    
    # Retornar resultado
    return tokens


print(get_tokens("I'm a super student for recommender systems!"))
print(get_tokens("First sentence. Seconde sentence."))

['im', 'a', 'super', 'student', 'for', 'recommender', 'systems']
['first', 'sentence', 'seconde', 'sentence']


En el código anterior, para ejecutar `nltk.word_tokenize()` era necesario tener descargado _punkt_. 

## Actividad 2

En función a las frases ingresadas y al resultado impreso, ¿Qué significa _Tokenizar_?

**Respuesta:** COMPLETAR

In [9]:
# A cada abstract le aplicamos la función de get_tokens
corpus_df['tokenized_abstract'] = corpus_df.abstract.map(get_tokens)
corpus_df.head(5)

,id,title,abstract,tokenized_abstract
0,100002,Nonlinear Shape Statistics in Mumford{Shah Bas...,We present a variational integration of nonlin...,"[we, present, a, variational, integration, of,..."
1,100007,On the Complexity of Equilibria,"We prove complexity, approximability, and inap...","[we, prove, complexity, approximability, and, ..."
2,100008,On QoS-Aware Publish-Subscribe,This position paper addresses the issue of sup...,"[this, position, paper, addresses, the, issue,..."
3,10001,Checking Mergeable Priority Queues,We present an efficient algorithm which can ch...,"[we, present, an, efficient, algorithm, which,..."
4,100012,Mobile Code Security by Java Bytecode Instrume...,Mobile code provides significant opportunities...,"[mobile, code, provides, significant, opportun..."


Ahora se tiene que generar un diccionario con todas las palabras del *corpus*. Se recomienda revisar la documentación de gensim y leer cómo usar los diccionarios: [corpora.dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html)

In [0]:
dict_file = './resources/dictionary.p'

if isfile(dict_file): # Verificar si existe el archivo
    dictionary = corpora.dictionary.Dictionary().load(dict_file)
    
else: # En otro caso, crear el archivo y guardarlo
    dictionary = corpora.dictionary.Dictionary(documents=corpus_df.tokenised_abstract.tolist())
    dictionary.save(dict_file)

In [11]:
# Texto original
print("Texto 1")
wrap(str(corpus_df.loc[0]["tokenized_abstract"]))


Texto 1


["['we', 'present', 'a', 'variational', 'integration', 'of',",
 "'nonlinear', 'shape', 'statistics', 'into', 'a', 'mumfordshah',",
 "'based', 'segmentation', 'process', 'the', 'nonlinear', 'statistics',",
 "'are', 'derived', 'from', 'a', 'set', 'of', 'training', 'silhouettes',",
 "'by', 'a', 'novel', 'method', 'of', 'density', 'estimation', 'which',",
 "'can', 'be', 'considered', 'as', 'an', 'extension', 'of', 'kernel',",
 "'pca', 'to', 'a', 'stochastic', 'framework']"]

In [12]:
# Texto pasado por el diccionario
print("Texto 1")
wrap(str(dictionary.doc2bow(corpus_df.loc[0]["tokenized_abstract"])))


Texto 1


['[(0, 1), (1, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9,',
 '1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (16, 1), (17, 1),',
 '(18, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26,',
 '2), (27, 1), (28, 1), (29, 1), (30, 5), (31, 1), (32, 1), (33, 1),',
 '(34, 1), (35, 4), (36, 1), (37, 1), (38, 1), (40, 1)]']

Cuando se hizo `dictionary.doc2bow` se transformó una lista de palabas a un contador de ellas. En donde cada tupla representa `(ID, cantidad de veces)` de modo que se reduce la cantidad de palabras del texto a información numerica. 

Por ejemplo, la tupla `(30, 5)` indica que la palabra con ID 30 está 5 veces en el texto. Revisando el texto podemos ver que la palabra **"a"** es la que está repetida 5 veces. Esto implica que **"a"** está asignada al ID 30.

Ahora aplicaremos esta función a cada texto del _dataset_.

In [13]:
corpus_df['bow'] = corpus_df.tokenized_abstract.map(dictionary.doc2bow)

corpus = corpus_df['bow'].tolist()

corpus_df.head(5)

,id,title,abstract,tokenized_abstract,bow
0,100002,Nonlinear Shape Statistics in Mumford{Shah Bas...,We present a variational integration of nonlin...,"[we, present, a, variational, integration, of,...","[(0, 1), (1, 1), (3, 1), (4, 1), (5, 1), (6, 1..."
1,100007,On the Complexity of Equilibria,"We prove complexity, approximability, and inap...","[we, prove, complexity, approximability, and, ...","[(4, 5), (7, 2), (8, 1), (10, 1), (30, 3), (35..."
2,100008,On QoS-Aware Publish-Subscribe,This position paper addresses the issue of sup...,"[this, position, paper, addresses, the, issue,...","[(1, 1), (4, 6), (7, 1), (16, 1), (22, 1), (27..."
3,10001,Checking Mergeable Priority Queues,We present an efficient algorithm which can ch...,"[we, present, an, efficient, algorithm, which,...","[(4, 7), (7, 2), (8, 1), (10, 1), (16, 1), (17..."
4,100012,Mobile Code Security by Java Bytecode Instrume...,Mobile code provides significant opportunities...,"[mobile, code, provides, significant, opportun...","[(4, 4), (5, 1), (7, 1), (10, 2), (16, 5), (22..."


# Tf-idf

Recordemos que Tf-idf es una medida numérica que expresa cuán relevante es una palabra para un documento en una colección. Ahora, dada la frecuencia de cada palabra en cada texto, se v a utilizar esta ténica para obtener tuplas de la forma `(ID, Tf-idf)` en donde ID será el ID de la palabra igual como estaba antes (por ejemplo **"a"** tiene ID 30) y Tf-Idf será el valor dado por este algoritmo a la palabra en cuestión.

In [14]:
tfidf_model_file = 'resources/tfidf_model.p'

if isfile(tfidf_model_file):
    tfidf_model = models.tfidfmodel.TfidfModel().load(tfidf_model_file)

else:
    tfidf_model = models.tfidfmodel.TfidfModel(corpus, dictionary=dictionary)
    tfidf_model.save(tfidf_model_file)

corpus_df['tf_idf'] = tfidf_model[corpus_df.bow.tolist()]
corpus_df.head(5)

,id,title,abstract,tokenized_abstract,bow,tf_idf
0,100002,Nonlinear Shape Statistics in Mumford{Shah Bas...,We present a variational integration of nonlin...,"[we, present, a, variational, integration, of,...","[(0, 1), (1, 1), (3, 1), (4, 1), (5, 1), (6, 1...","[(0, 0.19689725999527163), (1, 0.0861613877917..."
1,100007,On the Complexity of Equilibria,"We prove complexity, approximability, and inap...","[we, prove, complexity, approximability, and, ...","[(4, 5), (7, 2), (8, 1), (10, 1), (30, 3), (35...","[(4, 0.0033554011043417254), (7, 0.02333778550..."
2,100008,On QoS-Aware Publish-Subscribe,This position paper addresses the issue of sup...,"[this, position, paper, addresses, the, issue,...","[(1, 1), (4, 6), (7, 1), (16, 1), (22, 1), (27...","[(1, 0.06276351152911328), (4, 0.0049492930133..."
3,10001,Checking Mergeable Priority Queues,We present an efficient algorithm which can ch...,"[we, present, an, efficient, algorithm, which,...","[(4, 7), (7, 2), (8, 1), (10, 1), (16, 1), (17...","[(4, 0.0022699486545179476), (7, 0.01127724975..."
4,100012,Mobile Code Security by Java Bytecode Instrume...,Mobile code provides significant opportunities...,"[mobile, code, provides, significant, opportun...","[(4, 4), (5, 1), (7, 1), (10, 2), (16, 5), (22...","[(4, 0.001715799318906219), (5, 0.031751265629..."


## Generar recomendaciones: 
En esta sección se implementan las funciones necesarias para poder generar recomendaciones dado lo que un usuario ha consumido. De manera artificial, se "samplearán" 3 documentos aleatorios que representarán al usuario objetivo (`sample`). Luego tendrás que generar diferentes recomendaciones y evaluar los resultados.

In [15]:
# Random users
samples = corpus_df.sample(3)
samples_ids = []

for n, (ix, paper) in enumerate(samples.iterrows()):
    samples_ids.append(ix)
    idx, title, abstract, bow, tf_idf = paper[[
        'id', 'title', 'abstract', 'bow', 'tf_idf']]
    print('%d) %s' % (n+1, title))
    print('')
    print("\n".join(wrap(abstract)))
    print('\n')

1) On the (un)reality of Virtual Types

We show, mostly through detailed examples, that programming patterns
known to involve the notion of virtual types can be implemented
directly and concisely using parametric polymorphism. A significant
improvement we make over previous approaches is to allow related
classes to be defined independently. This solution is more flexible,
more general, and we believe, simpler than other type-safe solutions
previously proposed. This approach can be applied to several languages
with parametric polymorphism that can already type binary methods and
have structural object types. We conclude that parametric polymorphism
is always better than virtual types. Moreover, it avoids the
introduction of a new non-atomic primitive construct. Introduction The
programming patterns that lead to the introduction of virtual types
are not new in the object-oriented community. However, they recently
became famous for the typechecking challenges that they posed to some
recen

Lo anterior son 3 textos tomados al azar. Asumiremso que una persona vió estos 3 textos y ahora vamos a recomendarle 5 nuevos por cada documento.

In [0]:
# Recommendation functions

N = len(dictionary)


def to_sparse(matrix):
    return csr_matrix([
        gensim.matutils.sparse2full(row, length=N)
        for row in matrix
    ])


def make_recommendations(model, metric, neighbors):
    M = len(corpus)

    X = to_sparse(corpus_df[model].tolist())
    document_index = NearestNeighbors(
        n_neighbors=(neighbors + 1),
        algorithm='brute',
        metric=metric).fit(X)
    return document_index


def print_recommendations(indexes, model):
    for n, (ix, paper) in enumerate(samples.iterrows()):
        dists, neighbors = indexes.kneighbors([gensim.matutils.sparse2full(paper[model], length=N)])
        print(paper['title'])
        print('')
        print('Documentos cercanos: ')
        i = 1
        for neighbour in neighbors[0]:
            if ix != neighbour:
                line = str(i) + ". " + corpus_df.iloc[neighbour]['title']
                print(line)
                i += 1
        print('\n')

A continuación deberá utilizar las funciones implementadas anteriormente para generar nuevas recomendaciones variando los parámetros del modelo. **Agregue nuevas celdas para cada implementación y/o pregunta.**

In [17]:
# Recommendation example: TF-IDF
doc_idx = make_recommendations('tf_idf', 'euclidean', 5)
print_recommendations(doc_idx, 'tf_idf')

On the (un)reality of Virtual Types

Documentos cercanos: 
1. A Statically Safe Alternative to Virtual Types
2. Lightweight Parametric Polymorphism for Oberon
3. How to Make Ad-Hoc Polymorphism Less Ad Hoc
4. Notes on Sconing and Relators
5. The Cartesian Product Algorithm - Simple and Precise Type Inference of Parametric Polymorphism


Feudal Reinforcement Learning

Documentos cercanos: 
1. On-Line Q-Learning Using Connectionist Systems
2. Context-Based Policy Search: Transfer of Experience Across Problems
3. Hierarchical Learning in Stochastic Domains: Preliminary Results
4. Learning Control Composition in a Complex Environment
5. Problem Solving With Reinforcement Learning


Affective Computing

Documentos cercanos: 
1. Building Emotional Agents
2. Toward Agents that Recognize Emotion
3. Comprehension Processes During Large Scale Maintenance
4. Towards a Society of Affect-driven Agents
5. System Design by Composing Structures of Interacting Objects




# Stop words

A continuación, intentaremos mejorar los resultados obtenidos con LDA eliminando las *stopwords*. ¿Qué son las *stopwords*? Son palabras vacías, sin significado, que no aportan (de manera significativa) al sentido de una frase, como los artículos, pronombres, etc.

In [29]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    filtered_words = [
        word for word in text if word not in stopwords.words('english')
    ]
    return filtered_words

Ahora eliminamos los stopwords de los textos y volvemos a hacer todo el proceso pero con textos diferentes. Este proceso dura aproximadamente **5 minutos**

In [31]:
%%time
# Puede que se demore un poco esta celda
corpus_df['tokenized_abstract_without_stopwords'] = corpus_df.tokenized_abstract.map(remove_stopwords)

CPU times: user 1min 43s, sys: 10.9 s, total: 1min 54s
Wall time: 1min 54s


In [32]:
corpus_df.head(5)

,id,title,abstract,tokenized_abstract,bow,tf_idf,tokenized_abstract_without_stopwords
0,100002,Nonlinear Shape Statistics in Mumford{Shah Bas...,We present a variational integration of nonlin...,"[we, present, a, variational, integration, of,...","[(0, 1), (1, 1), (3, 1), (4, 1), (5, 1), (6, 1...","[(0, 0.19689725999527163), (1, 0.0861613877917...","[present, variational, integration, nonlinear,..."
1,100007,On the Complexity of Equilibria,"We prove complexity, approximability, and inap...","[we, prove, complexity, approximability, and, ...","[(4, 5), (7, 2), (8, 1), (10, 1), (30, 3), (35...","[(4, 0.0033554011043417254), (7, 0.02333778550...","[prove, complexity, approximability, inapproxi..."
2,100008,On QoS-Aware Publish-Subscribe,This position paper addresses the issue of sup...,"[this, position, paper, addresses, the, issue,...","[(1, 1), (4, 6), (7, 1), (16, 1), (22, 1), (27...","[(1, 0.06276351152911328), (4, 0.0049492930133...","[position, paper, addresses, issue, supporting..."
3,10001,Checking Mergeable Priority Queues,We present an efficient algorithm which can ch...,"[we, present, an, efficient, algorithm, which,...","[(4, 7), (7, 2), (8, 1), (10, 1), (16, 1), (17...","[(4, 0.0022699486545179476), (7, 0.01127724975...","[present, efficient, algorithm, check, answers..."
4,100012,Mobile Code Security by Java Bytecode Instrume...,Mobile code provides significant opportunities...,"[mobile, code, provides, significant, opportun...","[(4, 4), (5, 1), (7, 1), (10, 2), (16, 5), (22...","[(4, 0.001715799318906219), (5, 0.031751265629...","[mobile, code, provides, significant, opportun..."


In [33]:
corpus_df['bow_without_stopwords'] = corpus_df.tokenized_abstract_without_stopwords.map(dictionary.doc2bow)
corpus_df.head(5)

,id,title,abstract,tokenized_abstract,bow,tf_idf,tokenized_abstract_without_stopwords,bow_without_stopwords
0,100002,Nonlinear Shape Statistics in Mumford{Shah Bas...,We present a variational integration of nonlin...,"[we, present, a, variational, integration, of,...","[(0, 1), (1, 1), (3, 1), (4, 1), (5, 1), (6, 1...","[(0, 0.19689725999527163), (1, 0.0861613877917...","[present, variational, integration, nonlinear,...","[(0, 1), (1, 1), (3, 1), (6, 1), (9, 1), (11, ..."
1,100007,On the Complexity of Equilibria,"We prove complexity, approximability, and inap...","[we, prove, complexity, approximability, and, ...","[(4, 5), (7, 2), (8, 1), (10, 1), (30, 3), (35...","[(4, 0.0033554011043417254), (7, 0.02333778550...","[prove, complexity, approximability, inapproxi...","[(42, 1), (43, 1), (44, 1), (45, 1), (46, 1), ..."
2,100008,On QoS-Aware Publish-Subscribe,This position paper addresses the issue of sup...,"[this, position, paper, addresses, the, issue,...","[(1, 1), (4, 6), (7, 1), (16, 1), (22, 1), (27...","[(1, 0.06276351152911328), (4, 0.0049492930133...","[position, paper, addresses, issue, supporting...","[(1, 1), (90, 1), (91, 3), (92, 1), (93, 1), (..."
3,10001,Checking Mergeable Priority Queues,We present an efficient algorithm which can ch...,"[we, present, an, efficient, algorithm, which,...","[(4, 7), (7, 2), (8, 1), (10, 1), (16, 1), (17...","[(4, 0.0022699486545179476), (7, 0.01127724975...","[present, efficient, algorithm, check, answers...","[(29, 1), (85, 2), (95, 1), (133, 1), (134, 1)..."
4,100012,Mobile Code Security by Java Bytecode Instrume...,Mobile code provides significant opportunities...,"[mobile, code, provides, significant, opportun...","[(4, 4), (5, 1), (7, 1), (10, 2), (16, 5), (22...","[(4, 0.001715799318906219), (5, 0.031751265629...","[mobile, code, provides, significant, opportun...","[(25, 1), (29, 1), (103, 1), (108, 1), (172, 3..."


In [34]:
corpus = corpus_df['bow_without_stopwords'].tolist()

tfidf_model_file_without_stopwords = 'resources/tfidf_model.p'

if isfile(tfidf_model_file):
    tfidf_model_without_stopwords = models.tfidfmodel.TfidfModel().load(tfidf_model_file)

else:
    tfidf_model_without_stopwords = models.tfidfmodel.TfidfModel(corpus, dictionary=dictionary)
    tfidf_model_without_stopwords.save(tfidf_model_file_without_stopwords)

corpus_df['tf_idf_without_stopwords'] = tfidf_model_without_stopwords[corpus_df.bow_without_stopwords.tolist()]
corpus_df.head(5)

,id,title,abstract,tokenized_abstract,bow,tf_idf,tokenized_abstract_without_stopwords,bow_without_stopwords,tf_idf_without_stopwords
0,100002,Nonlinear Shape Statistics in Mumford{Shah Bas...,We present a variational integration of nonlin...,"[we, present, a, variational, integration, of,...","[(0, 1), (1, 1), (3, 1), (4, 1), (5, 1), (6, 1...","[(0, 0.19689725999527163), (1, 0.0861613877917...","[present, variational, integration, nonlinear,...","[(0, 1), (1, 1), (3, 1), (6, 1), (9, 1), (11, ...","[(0, 0.19859067348407708), (1, 0.0869024181966..."
1,100007,On the Complexity of Equilibria,"We prove complexity, approximability, and inap...","[we, prove, complexity, approximability, and, ...","[(4, 5), (7, 2), (8, 1), (10, 1), (30, 3), (35...","[(4, 0.0033554011043417254), (7, 0.02333778550...","[prove, complexity, approximability, inapproxi...","[(42, 1), (43, 1), (44, 1), (45, 1), (46, 1), ...","[(42, 0.1372259998572753), (43, 0.045914969188..."
2,100008,On QoS-Aware Publish-Subscribe,This position paper addresses the issue of sup...,"[this, position, paper, addresses, the, issue,...","[(1, 1), (4, 6), (7, 1), (16, 1), (22, 1), (27...","[(1, 0.06276351152911328), (4, 0.0049492930133...","[position, paper, addresses, issue, supporting...","[(1, 1), (90, 1), (91, 3), (92, 1), (93, 1), (...","[(1, 0.06604705077143881), (90, 0.172432977199..."
3,10001,Checking Mergeable Priority Queues,We present an efficient algorithm which can ch...,"[we, present, an, efficient, algorithm, which,...","[(4, 7), (7, 2), (8, 1), (10, 1), (16, 1), (17...","[(4, 0.0022699486545179476), (7, 0.01127724975...","[present, efficient, algorithm, check, answers...","[(29, 1), (85, 2), (95, 1), (133, 1), (134, 1)...","[(29, 0.020859058995752067), (85, 0.0447629771..."
4,100012,Mobile Code Security by Java Bytecode Instrume...,Mobile code provides significant opportunities...,"[mobile, code, provides, significant, opportun...","[(4, 4), (5, 1), (7, 1), (10, 2), (16, 5), (22...","[(4, 0.001715799318906219), (5, 0.031751265629...","[mobile, code, provides, significant, opportun...","[(25, 1), (29, 1), (103, 1), (108, 1), (172, 3...","[(25, 0.04074940143660829), (29, 0.02767893128..."


# Word Embeddings
En esta sección haremos recomendacion de textos médicos de [PubMed](https://www.ncbi.nlm.nih.gov/pubmed/) que han sido revisados por expertos. 

RESPONDER LAS SIGUIENTES PREGUNTAS: 
- ¿Que son word embeddings? ¿Cuál es la intuición?
- ¿Por qué son útiles para representar documentos?


In [18]:
# Descarga de recursos
!wget https://www.dropbox.com/s/gc3x9rp4gu2tmch/documents_w2vec.json.zip
!unzip documents_w2vec.json.zip

--2019-08-26 16:17:54--  https://www.dropbox.com/s/gc3x9rp4gu2tmch/documents_w2vec.json.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/gc3x9rp4gu2tmch/documents_w2vec.json.zip [following]
--2019-08-26 16:17:55--  https://www.dropbox.com/s/raw/gc3x9rp4gu2tmch/documents_w2vec.json.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2565c6c54653ef667fc2ef6a88.dl.dropboxusercontent.com/cd/0/inline/AnWilxwhQR7C_GiIUIXbD16kShnuXmhQaFzdLaGEx0mG4ILdJaXy_lq6mhVn4Rn4EL9GQYWtJ0jA8sk9Sds1tK29V_q79YfICRAYZ2HsXnzX9mIJC8KY0CRzwrBN4bd0uwU/file# [following]
--2019-08-26 16:17:55--  https://uc2565c6c54653ef667fc2ef6a88.dl.dropboxusercontent.com/cd/0/inline/AnWilxwhQR7C_GiIUIXbD16kShnuXmhQaFzdLaGEx0mG4ILdJaXy_lq6mhVn4Rn4EL9GQYWtJ0jA8s

In [19]:
# Descarga del dataset
!wget https://www.dropbox.com/s/1bxuw3uf3xwyrr7/pubmed_data.csv

--2019-08-26 16:18:57--  https://www.dropbox.com/s/1bxuw3uf3xwyrr7/pubmed_data.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/1bxuw3uf3xwyrr7/pubmed_data.csv [following]
--2019-08-26 16:18:57--  https://www.dropbox.com/s/raw/1bxuw3uf3xwyrr7/pubmed_data.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc98f7795cfff123cca542a3088a.dl.dropboxusercontent.com/cd/0/inline/AnUHlhG_FV6Ff4MHTBcLMkjmzT19dh_1PyDa6hp05gBPF7cqwsyAYGqNll5Fh1Jh3A81IeB770ZOzkQZzgYeoq9EolgQAr4U_OZMxvNRinpbwLxuLqvn_Tx-kKShSURtg6k/file# [following]
--2019-08-26 16:18:58--  https://uc98f7795cfff123cca542a3088a.dl.dropboxusercontent.com/cd/0/inline/AnUHlhG_FV6Ff4MHTBcLMkjmzT19dh_1PyDa6hp05gBPF7cqwsyAYGqNll5Fh1Jh3A81IeB770ZOzkQZzgYeoq9EolgQAr4U_OZMxvNRi

Podemos ver que este _dataet: contiene 4 columnas:
* **user_id**: identificador de cada usuario 
* **pid**: identificador de cada texto con su correlativo de PubMed. 
* **_title_**: título del documento, en este caso, de un _paper_
* **_abstract_**: primer párafo del _paper_ que es una representación abreviada, objetiva y precisa del contenido de un documento o recurso.

In [20]:
df = pd.read_csv('pubmed_data.csv')
df.head()

,user_id,pid,title,abstract
0,348892,3242144,Value of percutaneous transhepatic cholangiosc...,"Since July 1975, percutaneous transhepatic bil..."
1,348893,3242144,Value of percutaneous transhepatic cholangiosc...,"Since July 1975, percutaneous transhepatic bil..."
2,348894,3242144,Value of percutaneous transhepatic cholangiosc...,"Since July 1975, percutaneous transhepatic bil..."
3,343531,17347632,Endoscopic management of a large choledochocel...,Choledochocele or type III choledochal cyst is...
4,348892,17347632,Endoscopic management of a large choledochocel...,Choledochocele or type III choledochal cyst is...


In [0]:
# creamos diccionario de titulos y abstracts que utilizaremos despues
dict_title_abstract = {}

for pid, title, abstract in zip(df.pid, df.title, df.abstract):
  dict_title_abstract[pid] = {'title': title, 'abstract': abstract}

In [0]:
# cargamos diccionario de embeddings por cada documento (pre-procesado)
w2vec_vectors = json.load(open('documents_w2vec.json'))


creamos un objeto *gensim.keyedvectors* para hacer más eficiente la búsqueda de documentos similares

In [0]:
embedding_size = 300

doc2vec = gensim.models.keyedvectors.Word2VecKeyedVectors(embedding_size)
keys = list(w2vec_vectors.keys())
values = [
    w2vec_vectors[key]
    for key in keys
]
doc2vec.add(keys, values)

## Generar recomendaciones

función **find_similar** para encontrar documentos similares a un pid en particular que recibe id del documento y los topn documentos mas similares y retorna topn documentos más similares

In [0]:
def find_similar(pid, topn):
  results = []

  for id_, score in doc2vec.similar_by_vector(doc2vec[pid], topn=topn):
      results.append([id_, score, dict_title_abstract[int(id_)]['title'], dict_title_abstract[int(id_)]['abstract']])

  return pd.DataFrame(results[1:], columns = ['pid', 'score', 'title', 'abstract'])

In [25]:
find_similar('22508578', 10)

,pid,score,title,abstract
0,19246909,0.893572,Validation study of the Mini-Mental State Exam...,In view of the differing sensitivity and speci...
1,12211125,0.885023,The distribution of Mini-Mental State Examinat...,to describe normative data for the Mini-Mental...
2,1538203,0.880047,Statistical description of the Mini-Mental Sta...,The distribution of total Mini-Mental State Ex...
3,17221080,0.879309,Mini-Mental State Examination norms in a commu...,The objective of this study was to assess Mini...
4,19337986,0.876571,Reliability and validity of revised Turkish ve...,To evaluate the reliability and validity of th...
5,17936377,0.875227,An adaptation of the Korean mini-mental state ...,The mini-mental state examination (MMSE) is a ...
6,18983719,0.873318,Applicability of the Mini-mental State Examina...,The Mini-mental State Examination (MMSE) is a ...
7,16906312,0.872315,[Mini-Mental State Examination: psychometric c...,To assess the psychometric characteristics of ...
8,1601631,0.866430,Mini-Mental State Examination (MMSE): sensitiv...,The sensitivity of the Mini-Mental State Exami...


función **recommend** para recomendar a un usuario de acuerdo una muestra de documentos que ha leído.

In [0]:
def recommend(user_id, topn, sample_user):
  user_docs =  df[df.user_id==user_id]['pid'].sample(sample_user)

  results = []

  for pid in user_docs:

    for id_, score in doc2vec.similar_by_vector(doc2vec[str(pid)], topn=topn):

      if int(id_) in dict_title_abstract:
        results.append([id_, score, dict_title_abstract[int(id_)]['title'], dict_title_abstract[int(id_)]['abstract']])
    

  results = sorted(results, key = lambda x: int(x[1]))

  return pd.DataFrame(results[topn:], columns = ['pid', 'score', 'title', 'abstract']).head(10)

In [27]:
# documentos leidos por el usuario 
df[df.user_id==348892].sample(10)

,user_id,pid,title,abstract
175227,348892,1292768,[Percutaneous removal of residual calculi of t...,Percutaneous instrumentation under fluoroscopi...
153482,348892,22104745,Update in biliary endoscopy.,Biliary endoscopy has seen the development of ...
2962,348892,9171132,Long-term results after laparoscopic cholecyst...,Cholecystotomy has been suggested for symptoma...
247775,348892,16968186,Multilocular flank abscess due to stone migrat...,We report a case of a patient who presented wi...
4153,348892,22519190,Dietary habits as a risk factor of gallstone d...,Gallstone formation is a multifactorial diseas...
174710,348892,1670228,Results of laparoscopic cholecystectomy in a u...,This study analyzed the first 100 laparoscopic...
10031,348892,16391906,Pneumobilia: where to look for on hepatic MR i...,The presence of pneumobilia is a particular pr...
152076,348892,429188,[Relaparotomy or endoscopic surgery in complic...,The number of cases in bile duct surgery has c...
174654,348892,10972182,The significance of personality in pain from g...,The aim of the study was to investigate the as...
249609,348892,11475852,[Ductal calculi as a diagnostic and therapeuti...,Currently used techniques of CBD lithiasis tre...


In [28]:
recommend(user_id= 348892, topn= 10, sample_user = 5)

,pid,score,title,abstract
0,7970057,0.716025,[Diagnostic-operative laparoscopy. Our experie...,Diagnostic laparoscopy has been an integral pa...
1,23134185,0.708834,Surgical treatment of gastrinomas: a single-ce...,"Gastrinomas are rare neuroendocrine tumours, a..."
2,25400740,0.700779,Endoscopic interventional treatment for gastri...,Endoscopic Interventional Treatment is of litt...
3,17907973,0.690769,Single-center experience of laparoscopic chole...,The laparoscopic cholecystectomy procedure is ...
4,18756345,0.688988,Gallstone ileus: retrospective review of a sin...,Gallstone ileus is responsible for 1-3 percent...
5,16282958,0.979679,Endoscopy for bile duct stones.,In the era of laparoscopic cholecystectomy and...
6,9828727,0.957200,Choledochoduodenostomy for common bile duct st...,Among the techniques for dealing with common b...
7,7426929,0.950031,Endoscopic sphincterotomy for bile duct stones.,This paper reports the results of endoscopic s...
8,10955066,0.949220,[Laparoscopy for common bile duct stones].,We performed 75 laparoscopic cholecystectomies...
9,17701853,0.947592,Balloon sphincteroplasty for removing difficul...,Extraction of common bile duct stones at endos...


RESPONDER:
- ¿Qué problemas puede tener la recomendación basada en contenido?